# Eastmoney Collector

## Stock Emotion Collector for Guba

In [1]:
import re
import time
import random
import hashlib
import datetime
import requests
import pandas as pd
import akshare as ak
from tqdm import tqdm
from lxml import etree
from functools import wraps
from joblib import Parallel, delayed, Memory
from libs.utils import format_code, get_proxy, format_code, proxy_request

Use the utility function `get_proxy` to get some proxies for later use.

In [2]:
mem = Memory('./cache')
proxies = mem.cache(get_proxy)()

### Crawl for the Web

This part is dedicated to crawl the up or down possibility provided by investers

In [ ]:
def crawl_stock(code: str):
    today = datetime.datetime.today().date()
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15",
        "Referer": "http://guba.eastmoney.com/",
        "Host": "gubacdn.dfcfw.com"
    }
    code = format_code(code, '{market}{code}')
    url = f"http://gubacdn.dfcfw.com/LookUpAndDown/{code}.js"
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    res = eval(res.text.strip('var LookUpAndDown=').replace('null', f'"{today}"'))
    data = pd.Series(res['Data'])
    data['code'] = code
    return data

In [ ]:
# benchmark: 36s
today = datetime.datetime.today().strftime('%Y%m%d')
codes = ak.stock_zh_a_spot_em()['代码'].to_list()
datas = Parallel(n_jobs=-1, backend='threading')(delayed(crawl_stock)(code) for code in codes)
data = pd.concat(datas, axis=1).T
data = data.set_index('code').drop('Date', axis=1)
data = data.astype({"TapeZ": "float32", "TapeD": "float32", "TapeType": "uint8"})
data = pd.concat([data], keys=[pd.to_datetime(today)], names=['datetime', 'instrument'])
data.to_parquet(f'../data/derivative_indicators/guba_votes/{today}.parquet')

### Crawl the Comments

In [ ]:
def overview(code: str, page: int, try_times: int = 100):
    tries = 0
    while tries <= try_times:
        page = str(page)
        url = f"http://guba.eastmoney.com/list,{code},f_{page}.html"
        html = etree.HTML(proxy_request(url, proxies=proxies, verbose=False).text)
        read = html.xpath('//*[@id="articlelistnew"]/div[not(@class="dheader")]/span[1]/text()')
        comments = html.xpath('//*[@id="articlelistnew"]/div[not(@class="dheader")]/span[2]/text()')
        title = html.xpath('//*[@id="articlelistnew"]/div[not(@class="dheader")]/span[3]/a/text()')
        href = html.xpath('//*[@id="articlelistnew"]/div[not(@class="dheader")]/span[3]/a/@href')
        author = html.xpath('//*[@id="articlelistnew"]/div[not(@class="dheader")]/span[4]/a/font/text()')
        time = html.xpath('//*[@id="articlelistnew"]/div[not(@class="dheader")]/span[5]/text()')
        # we might still got unaligned data because the anti-crawl system
        try:
            data = pd.DataFrame({"read": read, "comments": comments, "title": title, "href": href, "author": author, "datetime": time})
            return data
        except:
            tries += 1
            print(f"try time {tries} for page {page} aligned failed, that's probably because the proxy ip is banned.")

In [ ]:
end_page = 100
results = Parallel(n_jobs=-1, backend='threading')(
    delayed(overview)('zssh000001', i) for i in range(1, end_page + 1))

In [ ]:
today = datetime.datetime.today()
year = today.year
month = today.month
for res in results:
    res['datetime'] = res['datetime'].map(lambda x: (str(year) if int(x[:2]) <= month else str(year - 1)) + '-' + x)
    res['datetime'] = pd.to_datetime(res['datetime'])
    if (res['datetime'] < str(year)).any():
        year -= 1
result = pd.concat(results, axis=0)
result = result.astype({"read": "uint16", "comments": "uint16"})
result[['read', 'comments']] = result[['read', 'comments']].replace({'万': '0000', '\.': ''}, regex=True).astype('uint16')
result.to_parquet(f'../data/derivative_indicators/guba_comments/{today.date}.parquet')

## Stock Financial Report Collector

In [ ]:
def get_balance_sheet(code):
    """Only returns numerical balance_sheet_data"""
    try:
        data = ak.stock_balance_sheet_by_report_em(symbol=code)
        if data.empty:
            return None
        data = data.loc[:, data.columns[data.astype('f8', 
            errors='ignore').dtypes != 'object'].union(['SECUCODE', 'REPORT_DATE', 'NOTICE_DATE'])]
        data = data.replace({None: np.nan})
        data = data.astype('f8', errors='ignore')
        data[['REPORT_DATE', 'NOTICE_DATE']] = data[['REPORT_DATE', 'NOTICE_DATE']].astype('datetime64[ns]')
        data = data.set_index('REPORT_DATE')
        data = data.reindex(pd.date_range(data.index.min(), data.index.max(), freq='q'))
        data.index.name = "REPORT_DATE"
        data = data.reset_index()
        data['SECUCODE'] = data['SECUCODE'][~data['SECUCODE'].isna()].iloc[0]
        return data
    except:
        return None

In [ ]:
codes = get_stock_code()
joblibres = Parallel(n_jobs=8)(delayed(get_balance_sheet)(code) for code in codes)